## Enunciado

Cada día es más frecuente la introducción de la ciencia de datos en el ámbito del derecho y la justicia. Un ejemplo bien conocido de ello es el sistema COMPAS (Correctional offender management profiling for alternative sanctions) que se usa en varios estados de los Estados Unidos para hacer una evaluación del riesgo de reincidencia de las personas detenidas. Una breve descripción del sistema puede verse en la página https://en.wikipedia.org/wiki/COMPAS_(software).

En este caso, proporciona un conjunto de datos en bruto con información de las evaluaciones (fichero compas-scores.csv) y la historia legal de unos 11 000 casos en los años 2013 y 2014 (se trata de uno de los ficheros originales utilizados en un análisis independiente del sistema COMPAS llevado a cabo por ProPublica, disponible en internet). Aunque el conjunto de datos contiene información adicional, para resolver las cuestiones planteadas en este caso son necesarios (aparte de algunos campos cuyo nombre es autoexplicativo) los siguientes campos:

* “compas_screening_date”: se refiere a la fecha en la que se realizó la evaluación
* “decile_score”: es un número, de 1 a 10 que indica el riesgo de reincidencia en general (a mayor riesgo, mayor número).
* “v_decile_score”: es un número de 1 a 10, potencialmente distinto del anterior, que indica el riesgo de reincidencia en delitos violentos. Al hacer la evaluación de un caso en COMPAS, se generan las dos puntuaciones (entre otras cosas).
* “is_recid”: indicación de si la persona es reincidente (en el tiempo en que se recogen datos: no hay información de si la persona es reincidente más allá de ciertas fechas, y es importante tener esto en cuenta para asegurarse de hacer comparaciones homogéneas).
* “r_offense_date”: fecha en la que se cometió el delito por el que se considera reincidente a la persona.
* “is_violent_recid”: indicación de si la persona es reincidente en un delito con violencia (las mismas consideraciones sobre fechas que para “is_recid” aplican aquí)".
* “vr_offense_date”: fecha en la que se cometió el delito violento que da lugar a la consideración de reincidente.

### Se pide

1. Cargar los datos y realizar un análisis exploratorio y una evaluación de la calidad de los datos necesarios para el resto del caso. Específicamente, evaluar la integridad, validez y actualidad de los datos y proponer estrategias de mitigación de los posibles problemas encontrados.
2. ¿Son los campos “is_recid” e “is_violent_recid” en este conjunto de datos adecuados para evaluar la precisión de las estimaciones de riesgo generadas por el sistema COMPAS? Si no es así, definir y calcular una feature que sí lo sea.
3. El umbral para establecer medidas preventivas de la reincidencia es de 7 en adelante. Dado este umbral, generar una tabla de contingencia, explicando qué caso se considera como “positivo” (y, por lo tanto, cuáles son los errores de tipo I y los errores de tipo II).
4. El sistema asigna, de media, evaluaciones de riesgo más altas a los hombres que a las mujeres, y a las personas de raza afroamericana que a las de raza caucásica. Sin embargo, también las tasas de reincidencia son más altas para esos colectivos, aunque no está claro que la asignación de riesgo sea “justa” o no. Mostrar estas diferencias mediante representaciones gráficas y utilizarlas para analizar si la asignación de evaluaciones es justa o no.
5. ¿Para qué tipo de riesgos, el de delitos generales o el de delitos violentos, tiene el sistema más capacidad predictiva?

In [1]:
# Importar librerías

import pandas as pd
import numpy as np
import altair as alt

In [2]:
# Cargar datos

df = pd.read_csv('compas-scores.csv', index_col= 'id',
                 parse_dates = ['compas_screening_date', 'r_offense_date', 'vr_offense_date'])

In [3]:
# Estructura de los datos

df.shape

(11757, 46)

In [4]:
# Primeros 5 registros 

df.head()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
id,,,,,,,,,,,,,,,,,,,,,
1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,...,NaT,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,...,NaT,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,...,NaT,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,...,NaT,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [5]:
# .head no nos permite observar por completo el contenido de las variables, visualizamos el primer registro de manera distinta para ver los valores de cada variable
df.iloc[0]

name                                   miguel hernandez
first                                            miguel
last                                          hernandez
compas_screening_date               2013-08-14 00:00:00
sex                                                Male
dob                                          1947-04-18
age                                                  69
age_cat                                 Greater than 45
race                                              Other
juv_fel_count                                         0
decile_score                                          1
juv_misd_count                                        0
juv_other_count                                       0
priors_count                                          0
days_b_screening_arrest                              -1
c_jail_in                           2013-08-13 06:03:42
c_jail_out                          2013-08-14 05:41:20
c_case_number                             130113

In [6]:
# Últimos 5 registros 

df.tail()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
id,,,,,,,,,,,,,,,,,,,,,
11753,patrick hamilton,patrick,hamilton,2013-09-22,Male,1968-05-02,47,Greater than 45,Other,0,...,NaT,NaN,Risk of Violence,1,Low,2013-09-22,Risk of Recidivism,3,Low,2013-09-22
11754,raymond hernandez,raymond,hernandez,2013-05-17,Male,1993-06-24,22,Less than 25,Caucasian,0,...,NaT,NaN,Risk of Violence,5,Medium,2013-05-17,Risk of Recidivism,7,Medium,2013-05-17
11755,dieuseul pierre-gilles,dieuseul,pierre-gilles,2014-10-08,Male,1981-01-24,35,25 - 45,Other,0,...,NaT,NaN,Risk of Violence,3,Low,2014-10-08,Risk of Recidivism,4,Low,2014-10-08
11756,scott lomagistro,scott,lomagistro,2013-12-03,Male,1986-12-04,29,25 - 45,Caucasian,0,...,NaT,NaN,Risk of Violence,2,Low,2013-12-03,Risk of Recidivism,3,Low,2013-12-03
11757,chin yan,chin,yan,2014-01-11,Male,1982-02-19,34,25 - 45,Asian,0,...,NaT,NaN,Risk of Violence,1,Low,2014-01-11,Risk of Recidivism,1,Low,2014-01-11


In [7]:
# Información general del dataset

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11757 entries, 1 to 11757
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   name                     11757 non-null  object        
 1   first                    11757 non-null  object        
 2   last                     11757 non-null  object        
 3   compas_screening_date    11757 non-null  datetime64[ns]
 4   sex                      11757 non-null  object        
 5   dob                      11757 non-null  object        
 6   age                      11757 non-null  int64         
 7   age_cat                  11757 non-null  object        
 8   race                     11757 non-null  object        
 9   juv_fel_count            11757 non-null  int64         
 10  decile_score             11757 non-null  int64         
 11  juv_misd_count           11757 non-null  int64         
 12  juv_other_count          11757 n

In [8]:
# La variable decile_score se encuentra duplicada, verificamos que los valores sean los mismos
df.loc[df['decile_score'] != df['decile_score.1']]

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
id,,,,,,,,,,,,,,,,,,,,,


In [9]:
# Los valores son los mismos, procedemos a borrar decile_score.1
df = df.drop(columns=['decile_score.1'])

In [10]:
#Análisis Estadístico de las variables numéricas

df.describe()

,age,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,num_r_cases,r_days_from_arrest,is_violent_recid,num_vr_cases,v_decile_score
count,11757.000000,11757.000000,11757.000000,11757.000000,11757.000000,11757.000000,10577.000000,11015.000000,11757.000000,0.0,2460.000000,11757.000000,0.0,11757.000000
mean,35.143319,0.061580,4.371268,0.076040,0.093561,3.082164,-0.878037,63.587653,0.253806,NaN,20.410569,0.075019,NaN,3.571489
std,12.022894,0.445328,2.877598,0.449757,0.472003,4.687410,72.889298,341.899711,0.558324,NaN,74.354840,0.263433,NaN,2.500479
min,18.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,-597.000000,0.000000,-1.000000,NaN,-1.000000,0.000000,NaN,-1.000000
25%,25.000000,0.000000,2.000000,0.000000,0.000000,0.000000,-1.000000,1.000000,0.000000,NaN,0.000000,0.000000,NaN,1.000000
50%,32.000000,0.000000,4.000000,0.000000,0.000000,1.000000,-1.000000,1.000000,0.000000,NaN,0.000000,0.000000,NaN,3.000000
75%,43.000000,0.000000,7.000000,0.000000,0.000000,4.000000,-1.000000,2.000000,1.000000,NaN,1.000000,0.000000,NaN,5.000000
max,96.000000,20.000000,10.000000,13.000000,17.000000,43.000000,1057.000000,9485.000000,1.000000,NaN,993.000000,1.000000,NaN,10.000000


In [11]:
# Valores de reincidencia con inconsistencias

df.is_recid.loc[df['is_recid'] == -1].count()

719

In [12]:
#Valores de riesgo de reincidencia fuera del rango indicado

print(f"Valores fuera de rango de la Variable v_decile_score: {df.v_decile_score.loc[df['v_decile_score'] < 1].count()}")
print(f"Valores fuera de rango de la Variable decile_score: {df.decile_score.loc[df['decile_score'] < 1].count()}")

Valores fuera de rango de la Variable v_decile_score: 5
Valores fuera de rango de la Variable decile_score: 15


**Parte de Pregunta Nro. 1 y Nro. 2:** Existen problemas de validez fáciles de observar, por ejemplo, la variable "is_recid" presenta 719 valores con -1, lo cual se considera incorrecto, ya que su reincidencia debería limitarse a valores de 0 y 1; 1 para si, 0 para no, aspecto que debe ser corregido mediante una nueva feature a partir de las fechas de reincidencia. Además v_decile_score y decile_score, de acuerdo con lo indicado en el caso solo deben presentar valores entre 1 y 10, pero existen 5 y 15 casos, respectivamente, con valores inferiores a 1.

In [13]:
#Análisis de las variables tipo object

df.describe(include = 'object')

,name,first,last,sex,dob,age_cat,race,c_jail_in,c_jail_out,c_case_number,...,r_jail_out,vr_case_number,vr_charge_degree,vr_charge_desc,v_type_of_assessment,v_score_text,v_screening_date,type_of_assessment,score_text,screening_date
count,11757,11757,11757,11757,11757,11757,11757,10577,10577,11015,...,2460,882,882,882,11757,11752,11757,11757,11742,11757
unique,11584,4058,5921,2,7800,3,6,10577,10517,11015,...,953,882,9,88,1,3,704,1,3,704
top,michael cunningham,michael,williams,Male,1994-01-24,25 - 45,African-American,2014-07-22 12:50:30,2014-02-12 10:41:00,13002537CF10A,...,2014-02-18,16001450CF10A,(M1),Battery,Risk of Violence,Low,2013-03-20,Risk of Recidivism,Low,2013-03-20
freq,4,264,145,9336,6,6649,5813,1,4,1,...,10,1,372,356,11757,7968,39,11757,6607,39


In [14]:
#Análisis de las variables fecha

df.describe(include = 'datetime')

,compas_screening_date,r_offense_date,vr_offense_date
count,11757,3703,882
unique,704,1090,599
top,2013-03-20 00:00:00,2014-12-08 00:00:00,2015-08-15 00:00:00
freq,39,12,6
first,2013-01-01 00:00:00,2013-01-03 00:00:00,2013-01-28 00:00:00
last,2014-12-31 00:00:00,2016-03-29 00:00:00,2016-03-13 00:00:00


**Parte de la Pregunta Nro. 1:**
De acuerdo con el rango de fechas del cual se dispone, considerando la utilización del sistema en la actualidad, sería necesario recavar datos más cercanos al 2024, dado que es muy posible que las evaluaciones del sistema COMPAS ya no sean inferibles al comportamiento actual que se presenta respecto a la reincidencia de las personas detenidas. Además, se tiene información sobre la reincidencia de los detenidos hasta 2016, siendo el periodo de los datos de reincidencia de aproximadamente de poco más de 3 años, con fecha incluida para solo 3703 casos y 882 casos violentos. Volvemos al tema de la importancia de una actualización respecto a los datos, para obtener información sobre reincidencia en años posteriores y con ello mejorar la proporción de casos positivos vs negativos.

In [15]:
# Fechas de reincidencia vs indicador de reincidencia
print(f"Casos reincidentes: {df.is_recid.loc[df['is_recid'] == 1].count()}")
print(f"Casos reincidentes: {df.is_violent_recid.loc[df['is_violent_recid'] == 1].count()}")

Casos reincidentes: 3703
Casos reincidentes: 882


In [16]:
# Validación de casos indicados como reincidentes sin fecha asignada
df.loc[(df['is_recid'] == 1) & (df['r_offense_date'].isnull())]

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,score_text,screening_date
id,,,,,,,,,,,,,,,,,,,,,


In [17]:
# Validación de casos indicados como reincidentes violentos sin fecha asignada
df.loc[(df['is_violent_recid'] == 1) & (df['vr_offense_date'].isnull())]

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,score_text,screening_date
id,,,,,,,,,,,,,,,,,,,,,


Se valida que existe concordancia en los datos, respecto a que los casos que incluyen fechas de reincidencia y reincidencia violenta efectivamente también incluyen la fecha del evento que justifica dicha clasificación.

In [18]:
#Variables que presentan valores nulos

Valores_Nulos = pd.DataFrame(df.isnull().sum(), columns=['Cantidad_Nulos'])
Valores_Nulos['Porcentaje'] = Valores_Nulos['Cantidad_Nulos'] / len(df)*100
Valores_Nulos['Variable'] = Valores_Nulos.index
Valores_Nulos['Tipo'] = df.dtypes
Orden = ['Variable', 'Tipo', 'Cantidad_Nulos', 'Porcentaje']
Valores_Nulos = Valores_Nulos.reindex(columns=Orden)
Valores_Nulos.reset_index(drop=True, inplace=True)
Valores_Nulos.loc[Valores_Nulos['Cantidad_Nulos'] > 0]

,Variable,Tipo,Cantidad_Nulos,Porcentaje
14,days_b_screening_arrest,float64,1180,10.036574
15,c_jail_in,object,1180,10.036574
16,c_jail_out,object,1180,10.036574
17,c_case_number,object,742,6.311134
18,c_offense_date,object,2600,22.114485
19,c_arrest_date,object,9899,84.196649
20,c_days_from_compas,float64,742,6.311134
22,c_charge_desc,object,749,6.370673
24,num_r_cases,float64,11757,100.000000
25,r_case_number,object,8054,68.503870


In [19]:
# Cantidad de variables con valores nulos

print(f'En total son {df.shape[1]} variables que componen el dataset')
print(f"de las cuales {Valores_Nulos.loc[Valores_Nulos['Cantidad_Nulos'] > 0].count()[0]} presentan valores nulos")

En total son 45 variables que componen el dataset
de las cuales 22 presentan valores nulos


In [20]:
# Eliminamos columnas que solo tienen valores nulos
df_depurado = df.drop(columns=Valores_Nulos.loc[Valores_Nulos['Cantidad_Nulos'] == len(df)].Variable.tolist())
df_depurado.shape

(11757, 43)

**Parte de la Pregunta Nro. 1**: Respecto a la Integridad, se han identificado columnas que nos aportan información alguna ya que se encuentran solo con valores nulos, por lo que han sido excluidas del dataset. Respecto a los datos iniciales, o sea, los generados en su primera detención, existen varios casos que no incluyen datos específicos, como los días de detención o la descripción del delito, que podrían ser relevantes para definir una posible reincidencia, sin embargo, los procentajes de valores nulos de estas dos características son entre 10% y 5%, por lo que en este caso lo mejor sería excluir estos casos. 

Otras variables iniciales incluyen porcentajes significativos de valores nulos, como lo son la fecha de arresto y la fecha del delito, por lo que lo mejor sería excluir ambas variables del dataset utilizado para las evaluaciones.

In [21]:
# Nueva Feature para "is_recid", como se indicó anteriormente, hay 719 casos que indican -1

df_depurado['is_recid'] = df_depurado['r_offense_date'].apply(lambda x: 1 if pd.notna(x) else 0)

df_depurado.is_recid.describe()

count    11757.000000
mean         0.314961
std          0.464520
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: is_recid, dtype: float64

**Parte de la Pregunta Nro. 2:** Se corrige los valores de la variable 'is_recid' de acuerdo con lo indicado anteriormente. La variable 'is_violent_recid' de acuerdo con lo observado si presenta un rango correcto (Valores de 0 para negativo, valores de 1 para positivo).

**Pregunta Nro. 3:**

Ahora evaluaremos la capacidad del sistema para predecir al menos algún tipo de reincidencia y determinar ambos tipos de error

In [22]:
# Agrupar en una nueva variable los casos indicados como reincidentes, sean violentos o no

df_depurado['is_recid_general'] = df_depurado.apply(lambda row: 1 if row['is_recid'] == 1 or row['is_violent_recid'] == 1 else 0, axis=1)

In [23]:
# Agrupar en una nueva variable el valor más alto entre decile_score y v_decile_score para hacer la comparación

df_depurado['decile_score_general'] = df_depurado.apply(lambda row: max(row['decile_score'], row['v_decile_score']), axis=1)

In [24]:
#Tabla de Contingencia

umbral = 7
is_recid_pred_general = df_depurado['decile_score_general'].apply(lambda x: 0 if x < umbral else 1).rename('Predición')

In [25]:
tabla_recid_general = pd.crosstab(is_recid_pred_general, df_depurado.is_recid_general)
tabla_recid_general

is_recid_general,0,1
Predición,,
0,6259,2060
1,1795,1643


In [26]:
#Explicación de las predicciones
print(f"Casos positivos correctamente predichos: {tabla_recid_general[1][1]}")
print(f"Casos negativos correctamente predichos: {tabla_recid_general[0][0]}")

print(f"Falsos positivos (Error tipo I):  {tabla_recid_general[0][1]}")
print(f"Falsos negativos (Error tipo II):  {tabla_recid_general[1][0]}")

Casos positivos correctamente predichos: 1643
Casos negativos correctamente predichos: 6259
Falsos positivos (Error tipo I):  1795
Falsos negativos (Error tipo II):  2060


In [27]:
# Métricas de Evaluación

from sklearn.metrics import classification_report

print(classification_report(df_depurado.is_recid_general, is_recid_pred_general, digits = 3))

              precision    recall  f1-score   support

           0      0.752     0.777     0.765      8054
           1      0.478     0.444     0.460      3703

    accuracy                          0.672     11757
   macro avg      0.615     0.610     0.612     11757
weighted avg      0.666     0.672     0.669     11757



**Pregunta Nro. 3:** De acuerdo con las metricas obtenidas, la capacidad de pronosticar casos de reincidencia por parte del sistema es prácticamente aleatoria, cerca del 50%, por lo que se considera un mal pronosticador.

**Pregunta Nro. 4:**

In [28]:
# Inicialmente es importante conocer la proporción de los casos según su género
data = df_depurado.groupby(['sex'])['name'].count().reset_index()

alt.Chart(data=data, width=350, height=200, title='Segmentación por Género').mark_bar()\
    .encode(
    x = alt.X('sex:N', sort='-y', title = None),
    y = alt.Y('name:Q', title = 'Cantidad'),
    tooltip = alt.Tooltip('name:Q', format='.2f'))\
    .configure_axis(labelAngle=0).interactive()

alt.Chart(...)

In [29]:
#Evaluamos la puntuaciones otorgadas por sexo para reincidencia no violenta
data = df_depurado.groupby(['sex'])['decile_score'].mean().reset_index()

alt.Chart(data=data, width=350, height=200, title='Puntuación Media por Género').mark_bar()\
    .encode(
    x = alt.X('sex:N', sort='-y', title = None),
    y = alt.Y('decile_score:Q', title = 'Puntuación Media'),
    tooltip = alt.Tooltip('decile_score:Q', format='.2f'))\
    .configure_axis(labelAngle=0).interactive()

alt.Chart(...)

In [30]:
#Evaluamos la reincidencia por sexo
data = df_depurado.groupby(['sex'])['is_recid'].sum().reset_index()

alt.Chart(data=data, width=350, height=200, title='Número de Reincidencias por Género').mark_bar()\
    .encode(
    x = alt.X('sex:N', sort='-y', title = None),
    y = alt.Y('is_recid:Q', title = 'Cantidad de Reincidencias'),
    tooltip = alt.Tooltip('is_recid:Q', format='.2f'))\
    .configure_axis(labelAngle=0).interactive()

alt.Chart(...)

In [31]:
#Evaluamos la puntuaciones otorgadas por sexo para reincidencia violenta
data = df_depurado.groupby(['sex'])['v_decile_score'].mean().reset_index()

alt.Chart(data=data, width=350, height=200, title='Puntuación Media (Violenta) por Género').mark_bar()\
    .encode(
    x = alt.X('sex:N', sort='-y', title = None),
    y = alt.Y('v_decile_score:Q', title = 'Puntuación Media'),
    tooltip = alt.Tooltip('v_decile_score:Q', format='.2f'))\
    .configure_axis(labelAngle=0).interactive()

alt.Chart(...)

In [32]:
#Evaluamos la reincidencia violenta por sexo
data = df_depurado.groupby(['sex'])['is_violent_recid'].sum().reset_index()

alt.Chart(data=data, width=350, height=200, title='Número de Reincidencias Violentas por Género').mark_bar()\
    .encode(
    x = alt.X('sex:N', sort='-y', title = None),
    y = alt.Y('is_violent_recid:Q', title = 'Cantidad de Reincid. Violentas'),
    tooltip = alt.Tooltip('is_violent_recid:Q', format='.2f'))\
    .configure_axis(labelAngle=0).interactive()

alt.Chart(...)

In [33]:
#Tabla Resumen
data_2 = df_depurado.groupby(['sex']).agg({'name':'count', 'is_recid':'sum', 'is_violent_recid':'sum'}).reset_index()
data_2 = data_2.rename(columns={'sex':'Sexo', 'name':'Casos', 'is_recid':'Reincidentes', 'is_violent_recid':'Reincidentes_Violentos'})
data_2['%_Reincidentes'] = data_2['Reincidentes'] / data_2['Casos'] *100
data_2['%_Reincidentes Violentos'] = data_2['Reincidentes_Violentos'] / data_2['Casos'] *100
Orden = ['Sexo', 'Casos', 'Reincidentes', '%_Reincidentes', 'Reincidentes_Violentos', '%_Reincidentes Violentos']
data_2 = data_2.reindex(columns=Orden)
data_2

,Sexo,Casos,Reincidentes,%_Reincidentes,Reincidentes_Violentos,%_Reincidentes Violentos
0,Female,2421,576,23.791822,117,4.832714
1,Male,9336,3127,33.494002,765,8.194087


De acuerdo con lo observado en las gráficas anteriores, la asignación de calificaciones más altas para hombres si es justa, ya que reinciden más recurrentemente que las mujeres de acuerdo a los datos proporcionados, considerando también que la proporción de recurrencia del total de hombres es mayor a la proporción de recurrencia del total de las mujeres.

In [34]:
# Inicialmente es importante conocer la proporción de los casos según su raza
data = df_depurado.groupby(['race'])['name'].count().reset_index()

alt.Chart(data=data, width=350, height=200, title='Segmentación por Raza').mark_bar()\
    .encode(
    x = alt.X('race:N', sort='-y', title = None),
    y = alt.Y('name:Q', title = 'Cantidad'),
    tooltip = alt.Tooltip('name:Q', format='.2f'))\
    .configure_axis(labelAngle=-90).interactive()

alt.Chart(...)

In [35]:
#Evaluamos la puntuaciones otorgadas por raza para reincidencia no violenta
data = df_depurado.groupby(['race'])['decile_score'].mean().reset_index()

alt.Chart(data=data, width=350, height=200, title='Puntuación Media por Raza').mark_bar()\
    .encode(
    x = alt.X('race:N', sort='-y', title = None),
    y = alt.Y('decile_score:Q', title = 'Puntuación Media'),
    tooltip = alt.Tooltip('decile_score:Q', format='.2f'))\
    .configure_axis(labelAngle=-90).interactive()

alt.Chart(...)

In [36]:
#Evaluamos la reincidencia por raza
data = df_depurado.groupby(['race'])['is_recid'].sum().reset_index()

alt.Chart(data=data, width=350, height=200, title='Número de Reincidencias por Raza').mark_bar()\
    .encode(
    x = alt.X('race:N', sort='-y', title = None),
    y = alt.Y('is_recid:Q', title = 'Cantidad de Reincidencias'),
    tooltip = alt.Tooltip('is_recid:Q', format='.2f'))\
    .configure_axis(labelAngle=-90).interactive()

alt.Chart(...)

In [37]:
#Evaluamos la puntuaciones otorgadas por raza para reincidencia violenta
data = df_depurado.groupby(['race'])['v_decile_score'].mean().reset_index()

alt.Chart(data=data, width=350, height=200, title='Puntuación Media (Violenta) por Raza').mark_bar()\
    .encode(
    x = alt.X('race:N', sort='-y', title = None),
    y = alt.Y('v_decile_score:Q', title = 'Puntuación Media'),
    tooltip = alt.Tooltip('v_decile_score:Q', format='.2f'))\
    .configure_axis(labelAngle=-90).interactive()

alt.Chart(...)

In [38]:
#Evaluamos la reincidencia violenta por raza
data = df_depurado.groupby(['race'])['is_violent_recid'].sum().reset_index()

alt.Chart(data=data, width=350, height=200, title='Número de Reincidencias Violentas por Raza').mark_bar()\
    .encode(
    x = alt.X('race:N', sort='-y', title = None),
    y = alt.Y('is_violent_recid:Q', title = 'Cantidad de Reincid. Violentas'),
    tooltip = alt.Tooltip('is_violent_recid:Q', format='.2f'))\
    .configure_axis(labelAngle=-90).interactive()

alt.Chart(...)

In [39]:
#Tabla Resumen
data_2 = df_depurado.groupby(['race']).agg({'name':'count', 'is_recid':'sum', 'is_violent_recid':'sum'}).reset_index()
data_2 = data_2.rename(columns={'race':'Raza', 'name':'Casos', 'is_recid':'Reincidentes', 'is_violent_recid':'Reincidentes_Violentos'})
data_2['%_Reincidentes'] = data_2['Reincidentes'] / data_2['Casos'] *100
data_2['%_Reincidentes Violentos'] = data_2['Reincidentes_Violentos'] / data_2['Casos'] *100
Orden = ['Raza', 'Casos', 'Reincidentes', '%_Reincidentes', 'Reincidentes_Violentos', '%_Reincidentes Violentos']
data_2 = data_2.reindex(columns=Orden)
data_2

,Raza,Casos,Reincidentes,%_Reincidentes,Reincidentes_Violentos,%_Reincidentes Violentos
0,African-American,5813,2175,37.416136,541,9.306726
1,Asian,58,11,18.965517,4,6.896552
2,Caucasian,4085,1089,26.658507,233,5.703794
3,Hispanic,1100,261,23.727273,58,5.272727
4,Native American,40,13,32.500000,6,15.000000
5,Other,661,154,23.298033,40,6.051437


De acuerdo con lo observado en las gráficas anteriores, la asignación de calificaciones más altas para raza afroamericana si es justa, ya que reinciden más recurrentemente que las otras razas de acuerdo a los datos proporcionados, considerando también que la proporción de recurrencia del total de afroamericanos es mayor a la proporción de recurrencia del total de cada una de las otras razas, a excepción de los nativos americanos, que aunque corresponden a una proporción pequeña del total de casosm tienen una tasa alta de reincidencia, sin embargo, hay otros factores que influyen en la evaluación, por lo que no podemos decir que la evaluación sea injusta respecto al tipo de raza.

**Pregunta Nro. 5:**

In [40]:
#Tabla de Contingencia Reincidentes

is_recid_pred = df_depurado['decile_score'].apply(lambda x: 0 if x < umbral else 1).rename('Predicción')

In [41]:
tabla_recid = pd.crosstab(is_recid_pred, df_depurado.is_recid)
tabla_recid

is_recid,0,1
Predicción,,
0,6449,2200
1,1605,1503


In [42]:
# Medidas de evaluación para Reincidentes

print(classification_report(df_depurado.is_recid, is_recid_pred, digits = 3))

              precision    recall  f1-score   support

           0      0.746     0.801     0.772      8054
           1      0.484     0.406     0.441      3703

    accuracy                          0.676     11757
   macro avg      0.615     0.603     0.607     11757
weighted avg      0.663     0.676     0.668     11757



In [43]:
#Tabla de Contingencia Reincidentes Violentos
is_violent_recid_pred = df_depurado['v_decile_score'].apply(lambda x: 0 if x < umbral else 1).rename('Predicción')

In [44]:
tabla_violent_recid = pd.crosstab(is_violent_recid_pred, df_depurado.is_violent_recid)
tabla_violent_recid

is_violent_recid,0,1
Predicción,,
0,9359,616
1,1516,266


In [45]:
# Medidas de evaluación para Reincidentes Violentos

print(classification_report(df_depurado.is_violent_recid, is_violent_recid_pred, digits = 3))

              precision    recall  f1-score   support

           0      0.938     0.861     0.898     10875
           1      0.149     0.302     0.200       882

    accuracy                          0.819     11757
   macro avg      0.544     0.581     0.549     11757
weighted avg      0.879     0.819     0.845     11757



**Pregunta Nro. 5:** De acuerdo con las métricas obtenidas, el sistema tiene una mejor capacidad predictiva para determinar la reincidencia no violenta.